# 유튜브 비디오 데이터 분석

이 Jupyter Notebook은 유튜브 데이터를 분석하는 데 사용됩니다.  
분석의 주 목적은 유튜브 데이터를 통해 연관어 및 유튜브 카테고리 분류를 위한 NER(명명된 개체 인식) 모델의 객체명 분석기를 활용하는 것입니다.  

여기서 사용된 학습 데이터는 수집한 유튜브 데이터의 일부 샘플로 구성되어 있습니다.  
본 코드에서는 이 샘플 데이터를 기반으로 GLiNER_ko 모델을 파인튜닝하고, 예측 결과를 보여줍니다.  

우리의 목표는 NER 모델이 객체명을 정확히 인식하는 것이 아니라, 명사를 올바르게 분류했는지를 확인하는 것입니다.


## 명사만 사용하여 임베딩한 이유

텍스트에서 명사만 사용하여 임베딩하는 이유는 명사가 주로 문장의 주요 의미를 전달하기 때문입니다. 조사를 비롯한 불필요한 단어들을 제거함으로써 중요한 정보에 집중할 수 있으며, 이는 분류 모델의 성능을 향상시키는 효과가 있습니다.  
명사만을 사용하면 다음과 같은 이점이 있습니다:
- **정보 밀도 증가**: 명사는 문맥의 핵심을 구성하므로, 모델이 중요한 정보에 집중할 수 있습니다.
- **잡음 감소**: 조사나 접속사와 같은 단어들을 제거하면 모델이 불필요한 정보를 학습하는 것을 방지할 수 있습니다.
- **차원의 축소**: 불필요한 단어를 줄임으로써 텍스트의 길이가 줄어들고, 더 효율적인 임베딩이 가능합니다.

In [18]:
import json
import random
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

import torch
from gliner import GLiNERConfig, GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset
from tqdm import tqdm

In [2]:
# JSON 파일을 불러와 데이터를 섞은 후 90%는 학습용, 10%는 테스트용으로 분할
train_path = "../data/video_sample_train_data.json"

with open(train_path, "r") as f:
    data = json.load(f)

print('Dataset size:', len(data))

random.shuffle(data)
print('Dataset is shuffled...')

train_dataset = data[:int(len(data)*0.9)]
test_dataset = data[int(len(data)*0.9):]

print('Dataset is splitted...')

Dataset size: 12
Dataset is shuffled...
Dataset is splitted...


In [3]:
# 지정된 모델 ID와 캐시 디렉토리에서 GLiNER 모델을 로드하여 GPU 또는 CPU에서 사용할 준비를 합니다.
model_id = "taeminlee/gliner_ko"
cache_dir = "../models"

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model = GLiNER.from_pretrained(model_id, cache_dir=cache_dir)

Fetching 4 files: 100%|██████████| 4/4 [00:00<?, ?it/s]
c:\Users\csu52\anaconda3\envs\csu\lib\site-packages\transformers\convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
# 메모리 효율성은 낮지만 원래 구현을 모방하여 성능을 개선하기 위해 모델 구성에 맞춰 데이터 수집기를 설정합니다.
data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

In [5]:
# 선택적으로 모델을 빠른 훈련을 위해 지정된 장치(GPU 또는 CPU)로 이동시키고, 완료 메시지를 출력합니다.
model.to(device)
print("done")

done


In [6]:
# 훈련 데이터셋의 크기와 배치 크기를 기반으로 총 에포크 수를 계산하고, 
# 다양한 하이퍼파라미터를 설정하여 모델 훈련을 위한 TrainingArguments 객체를 생성한 다음, 
# 이 인자를 사용하여 모델과 데이터셋을 포함한 Trainer 객체를 초기화하여 모델 훈련을 시작합니다.
num_steps = 500
batch_size = 8
data_size = len(train_dataset)
num_batches = data_size // batch_size
num_epochs = max(1, num_steps // num_batches)

training_args = TrainingArguments(
    output_dir="../models/train",
    learning_rate=5e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    focal_loss_alpha=0.75,
    focal_loss_gamma=2,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    save_steps = 100,
    save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
)

trainer.train()

c:\Users\csu52\anaconda3\envs\csu\lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 50%|█████     | 500/1000 [25:00<23:55,  2.87s/it]  

{'loss': 6.9878, 'grad_norm': 0.0007905024685896933, 'learning_rate': 5.555555555555557e-06, 'epoch': 250.0}


                                                  
 50%|█████     | 500/1000 [25:00<23:55,  2.87s/it]

{'eval_loss': 107.31719970703125, 'eval_runtime': 0.0921, 'eval_samples_per_second': 21.717, 'eval_steps_per_second': 10.859, 'epoch': 250.0}


100%|██████████| 1000/1000 [48:33<00:00,  2.78s/it]

...

                                                   
100%|██████████| 1000/1000 [48:33<00:00,  2.78s/it]

{'eval_loss': 107.05731964111328, 'eval_runtime': 0.0642, 'eval_samples_per_second': 31.158, 'eval_steps_per_second': 15.579, 'epoch': 500.0}


100%|██████████| 1000/1000 [48:42<00:00,  2.92s/it]

{'train_runtime': 2922.1135, 'train_samples_per_second': 1.711, 'train_steps_per_second': 0.342, 'train_loss': 3.532232666015625, 'epoch': 500.0}


TrainOutput(global_step=1000, training_loss=3.532232666015625, metrics={'train_runtime': 2922.1135, 'train_samples_per_second': 1.711, 'train_steps_per_second': 0.342, 'total_flos': 0.0, 'train_loss': 3.532232666015625, 'epoch': 500.0})

In [7]:
# 훈련된 모델을 지정된 체크포인트(1000번째 체크포인트)에서 로드하며, 이때 토크나이저도 함께 불러옵니다.
trained_model = GLiNER.from_pretrained("../models/train/checkpoint-1000", load_tokenizer=True)


config.json not found in C:\Users\csu52\work\dothis\dothis-ai-labs\1_youtube_video_data_preprocessing\models\train\checkpoint-1000
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# 훈련된 GLiNER 모델을 사용하여 주어진 텍스트에서 엔티티 예측을 수행하고, 예측된 엔티티와 해당 레이블을 출력합니다. 
# 사용된 레이블 목록에는 다양한 개인정보 및 관련 정보가 포함되어 있습니다. 예측 결과는 entities 리스트에 저장되며, 
# 각 엔티티의 텍스트와 레이블이 출력됩니다.

text = """
국힘당 문자 하나로 파장! 지지율 폭락 예정! 장철민, 청탁금지법? 권익위원장에게 물어보니...노종면의원 가만두지 않겠다!
"""

# Labels for entity prediction
labels = ["artifacts", "person", "animal", "CIVILIZATION", "organization", \
        "phone number", "address", "passport number", "email", "credit card number", \
        "social security number", "health insurance id number", 'Business/organization', \
        "mobile phone number", "bank account number", "medication", "cpf", "driver's license number", \
        "tax identification number", "medical condition", "identity card number", "national id number", \
        "ip address", "email address", "iban", "credit card expiration date", "username", \
        "health insurance number", "student id number", "insurance number", \
        "flight number", "landline phone number", "blood type", "cvv", \
        "digital signature", "social media handle", "license plate number", "cnpj", "postal code", \
        "passport_number", "vehicle registration number", "credit card brand", \
        "fax number", "visa number", "insurance company", "identity document number", \
        "national health insurance number", "cvc", "birth certificate number", "train ticket number", \
        "passport expiration date", "social_security_number", "EVENT", "STUDY_FIELD", "LOCATION", \
        "MATERIAL", "PLANT", "TERM", "THEORY", 'Analysis Requirement']

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


국힘당 => organization
장철민 => person
청탁금지법 => CIVILIZATION
권익위원장에게 => CIVILIZATION
노종면의원 => CIVILIZATION


In [19]:
# JSON 데이터 불러오기
with open('../data/test.json', 'r', encoding='utf-8') as file:
    test = json.load(file)

# 예측 결과와 실제 레이블 비교
total_score = 0

for i in tqdm(test):
    # Perform entity prediction
    entities = trained_model.predict_entities(i['text'], labels, threshold=0.055)
    result = [i['text'] for i in entities]
    # 점수 계산: 예측된 레이블이 실제 레이블에 있는 경우 1점, 없으면 0점
    score = sum(1 for label in i["labels"] if label in result)
    total_score += score

# 전체 점수 및 정확도 계산
total_labels = sum(len(item["labels"]) for item in test)  # 전체 레이블 수
accuracy = total_score / total_labels if total_labels > 0 else 0  # 정확도 계산

# 결과 출력
print(f"정확도: {accuracy:.2f}")

100%|██████████| 5/5 [00:01<00:00,  3.47it/s]

정확도: 0.52
